<div style='margin: 55px 0; text-align: center; font-size: 21px'>Подготовим данные к последующей работе с ними. </div>

In [1]:
import pandas as pd
import numpy as np
import re

from dataprep.clean import clean_headers
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

In [2]:
#  очистка телефона
def format_tel(tel):
    if type(tel) == str:
        if tel.isalpha() == False:
            tel = tel.removeprefix("+")
            tel = tel.removeprefix("7")
            tel = tel.removeprefix("8")     # remove leading +1 or 1
            tel = re.sub("[ =()-]", '', tel) # remove space, (), -
            tel = f"{tel[:3]}-{tel[3:6]}-{tel[6:]}"
        else:
            tel = np.nan
    else:
        tel = np.nan
    return tel

# Преобразуют дату
def date(col):
    if isinstance(col, str):
        chars = ['.', '-']
        str_val = col.translate(str.maketrans({ord(x): '' for x in chars}))
        val = datetime.strptime(str_val, "%d%m%Y").date()
    else:
        val = col
    return val


<div style='margin: 55px 0; text-align: center; font-size: 21px'>
    Прочтём файл.<p style='text-align: center;font-size: 15px'>Продублируем файл,  переименуем колонки.</p>
</div>

In [3]:
df_origin = pd.read_excel('f/data.xlsx', usecols=lambda x: 'Unnamed' not in x)

In [4]:
df = df_origin.copy()

In [5]:
df.head(5)

,Name participant,AGE,weight__,Date,email Adress,contact_,money,Identifier
0,Katy,12.0,32.5,31.12.2013,abby@gmail.com,89664352265,900,578209
1,Abby,33.0,47.1,02.-11-2013,scott@gmail.com,NaN,900,669759
2,Scott,33.0,47.1,5.11.2013,NaN,+7 888 666 45 65,1.000.000руб,725068
3,Scott,56.0,55.2,2013-11-09 00:00:00,scott@gmail.com,79166872355,1000,603343
4,Scott2,NaN,NaN,23.11.2013,test@abc.com,8 987 546-87-66,1000000,799674


In [6]:
df = clean_headers(df, case="title")

Column Headers Cleaning Report:
	7 values cleaned (87.5%)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name Participant  66 non-null     object 
 1   Age               50 non-null     float64
 2   Weight            57 non-null     float64
 3   Date              65 non-null     object 
 4   Email Adress      59 non-null     object 
 5   Contact           65 non-null     object 
 6   Money             63 non-null     object 
 7   Identifier        66 non-null     int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 4.2+ KB


In [8]:
df.dtypes

Name Participant     object
Age                 float64
Weight              float64
Date                 object
Email Adress         object
Contact              object
Money                object
Identifier            int64
dtype: object

In [9]:
df.describe()

,Age,Weight,Identifier
count,50.00000,57.000000,66.000000
mean,41.60000,49.740351,749168.348485
std,16.65006,7.875184,118268.759778
min,12.00000,32.500000,578209.000000
25%,33.00000,47.100000,635118.250000
50%,56.00000,55.200000,733481.500000
75%,56.00000,55.200000,832831.750000
max,56.00000,55.200000,964825.000000


<div style='margin: 33px 0; text-align: center; font-size: 16px'>
    Получим типы данных контента в каждой колонки.<br>Далее уберём ненужные колонки из файла, установим индексы строк.<p style='text-align: center;font-size: 15px'></p>
</div>

In [10]:
dict_types = {}

for name, values in df.iteritems ():
    type_ = values.apply(type)
    set_ = set(list(type_))
    dict_types[f'{name}'] = list(set_)

dict_types

{'Name Participant': [str],
 'Age': [float],
 'Weight': [float],
 'Date': [str, float, datetime.datetime],
 'Email Adress': [str, float],
 'Contact': [str, float, int],
 'Money': [str, float, int],
 'Identifier': [int]}

In [11]:
to_drop = ['Age', 'Weight']

df.drop(to_drop, inplace=True, axis=1)

In [12]:
df['Identifier'].is_unique

True

In [13]:
df.set_index('Identifier', inplace=True)

<div style='margin: 55px 0; text-align: center; font-size: 16px'>
    Почистим значения номеров телефона, также значения оплаты. Откорректируем дату.<p style='text-align: center;font-size: 15px'></p>
</div>

In [14]:
df['Contact'] = df['Contact'].apply(format_tel)

In [15]:
df["Money"] = df["Money"].replace(r"[^\d]", "", regex=True).apply(pd.to_numeric)

In [16]:
df['Date'] = df['Date'].apply(date)

In [17]:
df.head(3)

,Name Participant,Date,Email Adress,Contact,Money
Identifier,,,,,
578209,Katy,2013-12-31,abby@gmail.com,966-435-2265,900.0
669759,Abby,2013-11-02,scott@gmail.com,NaN,900.0
725068,Scott,2013-11-05,NaN,888-666-4565,1000000.0


<div style='margin: 55px 0; text-align: center; font-size: 16px'>
     Переходим к отсутствующим значениям.<p style='text-align: center;font-size: 14px'>Получим отсутствующие значения из строк, заменим NaN, либо избавимся от строк с их содержанием.</p>
</div>

In [18]:
df=df.replace({'NULL':np.NaN})

In [19]:
df.isna().sum()

Name Participant    0
Date                1
Email Adress        7
Contact             5
Money               3
dtype: int64

In [20]:
df=df.dropna(subset=['Date','Contact'])

In [21]:
df['Money'].fillna((df['Money'].mean()), inplace=True)

In [22]:
df['Email Adress'] = df['Email Adress'].replace(np.nan, '/NA', regex=True)

In [23]:
df.isnull().values.any()

False

In [30]:
df.head()

,Name Participant,Date,Email Adress,Contact,Money
Identifier,,,,,
578209,Katy,2013-12-31,abby@gmail.com,966-435-2265,900.0
725068,Scott,2013-11-05,/NA,888-666-4565,1000000.0
799674,Scott2,2013-11-23,test@abc.com,987-546-8766,1000000.0
661334,Tony,2013-11-26,/NA,912-112-5544,-54400.0
958694,Scott,2013-11-09,scott@gmail.com,966-334-6654,900.0


<div style='margin: 55px 0; text-align: center; font-size: 16px'>
     Данные готовы к основному анализу.<p style='text-align: center;font-size: 14px'>Экспортируем их.</p>
</div>

In [24]:
df.to_excel('f/finished.xlsx')